In [1]:
import os
import pylidc as pl
from pylidc.utils import consensus
from configparser import ConfigParser
import matplotlib.pyplot as plt
from statistics import median_high
import numpy as np
import pandas as pd
from pathlib import Path
import preprocessing
from tqdm import tqdm

In [2]:
DataSet_root = "./LIDC-IDRI"
Processed_data_dir = './data'
IMAGE_DIR = '/IMAGE'
MASK_DIR = '/MASK'
mask_thresh = 8

In [ ]:
for patient in patients:
    pid = patient
    scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()
    
    nodule_annotations = scan.cluster_annotations()
    vol = scan.to_volume()
    patient_img_dir = Processed_data_dir + IMAGE_DIR + '/' + pid
    patient_mask_dir = Processed_data_dir + MASK_DIR + '/' + pid
    
    if(len(nodule_annotations)>0):
        for i,nodule in enumerate(nodule_annotations):
            print("Index{} Nodule{}".format(i,*nodule))
     

In [18]:
class prep_ds:
    DataSet_root = "./LIDC-IDRI"
    Processed_data_dir = './data'
    IMAGE_DIR = Processed_data_dir+ "/Image"
    MASK_DIR = Processed_data_dir +"/Mask"
    Clean_IMAGE_DIR =Processed_data_dir + '/CleanIMAGE'
    Clean_MASK_DIR = Processed_data_dir +'/CleanMASK'
    mask_thresh = 8

    dataFrame = pd.DataFrame(index=[],columns=['patient_id','nodule_no','slice_no','original_image','mask_image','malignancy','is_cancer','is_clean'])
    def save_dataFrame(self,meta_list):
      #saves list to csv
        tmp = pd.Series(meta_list,index=['patient_id','nodule_no','slice_no','original_image','mask_image','malignancy','is_cancer','is_clean'])
        self.dataFrame = self.dataFrame.append(tmp,ignore_index=True)
    def calc_malignancy(self,nodule):
        list_malignancy = []
        for annotation in nodule:
            list_malignancy.append(annotation.malignancy)
        malignancy = median_high(list_malignancy)
        if(malignancy>3):
            return malignancy,True
        if(malignancy<3):
            return malignancy,False
        else:
            return malignancy,"Ambiguous"

    
    def prepare_dataset(self):
            if not os.path.exists(self.IMAGE_DIR):
                os.makedirs(self.IMAGE_DIR)
            if not os.path.exists(self.MASK_DIR):
                os.makedirs(self.MASK_DIR)
            if not os.path.exists(self.Clean_IMAGE_DIR):
                os.makedirs(self.Clean_IMAGE_DIR)
            if not os.path.exists(self.Clean_MASK_DIR):
                os.makedirs(self.Clean_MASK_DIR)
            patients = os.listdir(DataSet_root)

            for patient in tqdm(patients):
                pid = patient
              
                scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()

                nodule_annotations = scan.cluster_annotations()
                vol = scan.to_volume()
      
                patient_img_dir = Processed_data_dir + IMAGE_DIR + '/' + pid
                patient_mask_dir = Processed_data_dir + MASK_DIR + '/' + pid
                Path(patient_img_dir).mkdir(parents=True, exist_ok=True)
                Path(patient_mask_dir).mkdir(parents=True, exist_ok=True)


                if(len(nodule_annotations)>0):

                    #nodules present
                    for i,nodule in enumerate(nodule_annotations):

                        mask,cbbox,masks = pl.utils.consensus(nodule,0.5,512)
                        lung_np = vol[cbbox]
                        malignancy,cancer_label = self.calc_malignancy(nodule)
                        for mask_slice in range(mask.shape[2]):

                            if(np.sum(mask[:,:,mask_slice])<=mask_thresh):
                                continue
                            segmented_lung = preprocessing.preprocess_image(lung_np[:,:,mask_thresh])
                            nodule_name =  "Nodule"+ str(i) +"Slice" +  str(mask_slice)
                            mask_name =  "Mask"+ str(i) +"Slice" +  str(mask_slice)
                            # to save pid nodule index slice index mask name malignancy cancer label is data clean of possible cancer
                            meta_list = [pid[-4:],i,mask_slice,patient_img_dir+ '/' +nodule_name,patient_mask_dir+ '/' +mask_name,malignancy,cancer_label,False]
                            self.save_dataFrame(meta_list)
                            print("Writing image to",patient_img_dir)
                            np.save(patient_img_dir+ '/' +nodule_name,segmented_lung)
                            np.save(patient_mask_dir+ '/' +mask_name,mask[:,:,mask_slice])
                else:
                    print("no nodules",pid)
                    cleanpatient_img_dir = self.Processed_data_dir + self.Clean_IMAGE_DIR + '/' + pid
                    cleanpatient_mask_dir = self.Processed_data_dir + self.Clean_MASK_DIR + '/' + pid
                    Path(cleanpatient_img_dir).mkdir(parents=True, exist_ok=True)
                    Path(cleanpatient_mask_dir).mkdir(parents=True, exist_ok=True)
                 
                    for slice in range(vol.shape[2]):
                        if(slice>50):
                            break
                        segmented_lung = preprocessing.preprocess_image(lung_np[:,:,mask_thresh])
                        lung_mask = np.zeros_like(segmented_lung)
                        nodule_name = "Nodule"+ str(i) +"Slice" +  str(slice)
                        mask_name = "Mask"+ str(i) +"Slice" +  str(slice)
                        meta_list = [pid[-4:],i,mask_slice,cleanpatient_img_dir+ '/' +nodule_name,cleanpatient_img_dir+ '/' +mask_name,0,False,True]
                        np.save(cleanpatient_img_dir+ '/' +nodule_name,segmented_lung)
                        np.save(cleanpatient_img_dir+ '/' +mask_name,mask[:,:,mask_slice])                    
                    
                    
                print("Saving CSV")
                self.dataFrame.to_csv(self.Processed_data_dir + "/dataset.csv",index = False)
                    
                    



                
               
            
            
            
            

In [19]:
ds = prep_ds()
ds.prepare_dataset()

  0%|                                                                                                                                                                                                                                                                                              | 0/30 [00:00<?, ?it/s]

Loading dicom files ... This may take a moment.
Writing image to ./data/IMAGE/LIDC-IDRI-0001
Writing image to ./data/IMAGE/LIDC-IDRI-0001
Writing image to ./data/IMAGE/LIDC-IDRI-0001
Writing image to ./data/IMAGE/LIDC-IDRI-0001
Writing image to ./data/IMAGE/LIDC-IDRI-0001
Writing image to ./data/IMAGE/LIDC-IDRI-0001
Writing image to ./data/IMAGE/LIDC-IDRI-0001


  3%|█████████▎                                                                                                                                                                                                                                                                            | 1/30 [00:11<05:41, 11.78s/it]

Writing image to ./data/IMAGE/LIDC-IDRI-0001
Saving CSV
Loading dicom files ... This may take a moment.
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0002
Writing image to ./data/IMAGE/LIDC-IDRI-0

  7%|██████████████████▌                                                                                                                                                                                                                                                                   | 2/30 [00:35<08:50, 18.93s/it]

...

Saving CSV
Loading dicom files ... This may take a moment.
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029
Writing image to ./data/IMAGE/LIDC-IDRI-0029


 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊         | 29/30 [07:28<00:17, 17.42s/it]

Writing image to ./data/IMAGE/LIDC-IDRI-0029
Saving CSV
Loading dicom files ... This may take a moment.
Writing image to ./data/IMAGE/LIDC-IDRI-0030
Writing image to ./data/IMAGE/LIDC-IDRI-0030
Writing image to ./data/IMAGE/LIDC-IDRI-0030
Writing image to ./data/IMAGE/LIDC-IDRI-0030


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [07:34<00:00, 15.14s/it]

Writing image to ./data/IMAGE/LIDC-IDRI-0030
Saving CSV
